In [33]:
import pandas as pd
%run C:\Users\sgasmi\Notebooks\Column_autosize.ipynb

## Chargement des données

In [2]:
UO_Annuaire = pd.read_csv(r'C:\Users\sgasmi\Desktop\Extractions\organizationNodes.csv', sep=";")
UO_PN = pd.read_excel(r"C:\Users\sgasmi\Desktop\Export Python\Referentiel UO.xlsx")

## Préparation des données

### Extraction des UO ayant pour parent final FRA

In [3]:
uid_fra="297A37C12A56CA16C37D63"

In [4]:
# Niveau 1
UO_Annuaire.loc[(UO_Annuaire["cn"]=="FRA")  & (UO_Annuaire["arevalevelnumber"]==1) , "FRA"]=1

# Niveau 2
UO_Annuaire.loc[UO_Annuaire["owner"]==uid_fra, "FRA"]=1
parent_lv2 = UO_Annuaire.loc[(UO_Annuaire["FRA"]==1) & (UO_Annuaire["owner"].isna()==False),"uid"]

# Niveau 3
UO_Annuaire.loc[UO_Annuaire["owner"].isin(parent_lv2)==True, "FRA"]=1
parent_lv3 =UO_Annuaire.loc[(UO_Annuaire["FRA"]==1) & (UO_Annuaire["owner"].isna()==False),"uid"]

#Niveau 4
UO_Annuaire.loc[UO_Annuaire["owner"].isin(parent_lv3)==True, "FRA"]=1

parent_lv4 =UO_Annuaire.loc[(UO_Annuaire["FRA"]==1) & (UO_Annuaire["owner"].isna()==False),"uid"]

#Niveau 5
UO_Annuaire.loc[UO_Annuaire["owner"].isin(parent_lv4)==True, "FRA"]=1


In [5]:
UO_Annuaire = UO_Annuaire.loc[UO_Annuaire["FRA"]==1]

### Ajout de l'UO mère dans UO_Annuaire

In [6]:
mother= UO_Annuaire[['uid', 'cn']]
UO_Annuaire = UO_Annuaire.merge(mother, left_on="owner", right_on="uid", how='left')

In [7]:
UO_Annuaire["arevalevelnumber"] = UO_Annuaire["arevalevelnumber"].apply(lambda x: "L"+str(x))

In [8]:
UO_Annuaire = UO_Annuaire.rename({"cn_y":"Label_mother", "cn_x": "Label", "owner_x": "uid_mother", "uid_x":"uid", "uid_y":"uid_mother"}, axis=1)

In [9]:
UO_PN.loc[UO_PN["UO mère"]=="FRAMATOME","UO mère"]="FRA"

In [10]:
UO_PN.loc[UO_PN["Nom de l'unité organisationnelle"]=="FRAMATOME"]="FRA"
UO_PN.loc[UO_PN["Nom de l'unité organisationnelle"]=="FRA", "Niveau"]="L1"

### Suppression des niveau 6,7,8 dans PN

In [11]:
UO_PN = UO_PN.loc[UO_PN["Niveau"].isin(["L6", "L7", "L8"])==False]

## UO à Créer

In [12]:
UO_to_add = UO_PN.loc[UO_PN["Nom de l'unité organisationnelle"].isin(UO_Annuaire["Label"])==False]

In [13]:
UO_to_add = UO_to_add[["Nom de l'unité organisationnelle", "Niveau", "Description", "UO mère"]]

In [14]:
UO_to_add = UO_to_add.sort_values("Nom de l'unité organisationnelle")

## Recherche des UO à supprimer

In [15]:
UO_to_delete = UO_Annuaire.loc[UO_Annuaire["Label"].isin(UO_PN["Nom de l'unité organisationnelle"])==False]

In [16]:
UO_to_delete = UO_to_delete[["uid", "Label", "arevalevelnumber"]]

In [17]:
UO_to_delete = UO_to_delete.sort_values("arevalevelnumber")

## UO à modifier 

In [18]:
UO_Annuaire = UO_Annuaire.merge(UO_PN[["Nom de l'unité organisationnelle", "Niveau", "UO mère"]], left_on="Label", right_on="Nom de l'unité organisationnelle", how="left")

In [19]:
UO_Annuaire.columns

Index(['arevalevelnumber', 'Label', 'uid', 'owner', 'labelFR', 'labelEN',
       'manager', 'FRA', 'uid_mother', 'Label_mother',
       'Nom de l'unité organisationnelle', 'Niveau', 'UO mère'],
      dtype='object')

In [20]:
UO_Annuaire = UO_Annuaire.rename({"arevalevelnumber":"Level_Annuaire", "Label_mother":"Mère_Annuaire","Nom de l'unité organisationnelle": "PN_Label", "Niveau": "PN_Level", "UO mère":"PN_UO_mère"}, axis=1)

In [21]:
UO_Annuaire["Ecart Level"] = UO_Annuaire.apply(lambda x: 1 if (x["Label"]==x["PN_Label"]) and (x["Level_Annuaire"] != x["PN_Level"]) else 0, axis=1)

In [22]:
UO_Annuaire["Ecart mère"] = UO_Annuaire.apply(lambda x: 1 if (x["Label"]==x["PN_Label"]) and (x["Mère_Annuaire"] != x["PN_UO_mère"]) else 0, axis=1)

In [23]:
Ecart = UO_Annuaire.loc[(UO_Annuaire["Ecart Level"]==1) | (UO_Annuaire["Ecart mère"]==1)]

In [24]:
Ecart = Ecart[["uid", "Label", "Ecart Level", "Ecart mère", "Level_Annuaire", "PN_Level", "Mère_Annuaire", "PN_UO_mère"]]

In [25]:
Ecart["PN_Level_mère"] = Ecart.apply(lambda x: "L" + str(int(x["PN_Level"][1])-1),axis=1)
Ecart["PN_Level_mère"] = Ecart["PN_Level_mère"].apply(lambda x: "L1" if x=="L0" else x)

In [26]:
mother= UO_Annuaire[['uid', 'Label', 'Level_Annuaire']]

In [27]:
Ecart = Ecart.merge(mother, right_on=["Label", "Level_Annuaire"] , left_on=["PN_UO_mère","PN_Level_mère"] , how="left")

In [28]:
Ecart = Ecart.drop(["Label_y", "Level_Annuaire_y"], axis=1)

In [29]:
Ecart = Ecart.rename( {"Level_Annuaire_x":"Level_Annuaire"}, axis=1)

In [30]:
Ecart = Ecart.rename({'uid_x':'uid', 'uid_y': "PN_UO_mère à ajouter","Label_x":"Label"}, axis=1)
Ecart = Ecart[["uid", "Label", "Ecart Level", "Ecart mère", "Level_Annuaire", "PN_Level", "Mère_Annuaire", "PN_UO_mère", "PN_UO_mère à ajouter", "PN_Level_mère"]]
Ecart = Ecart.sort_values("PN_Level")

## Recherche Niveau différent

## Export

In [31]:
with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\Update UO annuaire.xlsx') as writer:
    if len(UO_to_add)>0:
        UO_to_add.to_excel(writer, sheet_name='UO à ajouter', index=False)
    if len(UO_to_delete)>0:    
        UO_to_delete.to_excel(writer, sheet_name='UO à supprimer', index=False)
    if len(Ecart)>0:
        Ecart.to_excel(writer, sheet_name='Modification à effectuer', index=False)   

In [34]:
resize_column([UO_to_add,UO_to_delete, Ecart ], ["UO à ajouter", "UO à supprimer", "UO à modifier"], r'C:\Users\sgasmi\Desktop\Export Python\Update UO annuaire.xlsx' )